# TensorFlow Dataset

In [1]:
import tensorflow as tf

In [2]:
tf.version.VERSION

'2.0.0-dev20190126'

## Datasets playground

In [3]:
import tensorflow_datasets as tfds

# See available datasets
print(tfds.list_builders())

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [0]:
# Construct a tf.data.Dataset
dataset = tfds.load(name="fashion_mnist", data_dir='~/tensorflow_datasets')

In [0]:
ls -la /root/tensorflow_datasets/fashion_mnist/1.0.0

total 38428
drwxr-xr-x 2 root root    4096 Feb  5 17:58 ./
drwxr-xr-x 3 root root    4096 Feb  5 17:58 ../
-rw-r--r-- 1 root root    3372 Feb  5 17:58 dataset_info.json
-rw-r--r-- 1 root root 5630824 Feb  5 17:57 fashion_mnist-test.tfrecord-00000-of-00001
-rw-r--r-- 1 root root 3370332 Feb  5 17:57 fashion_mnist-train.tfrecord-00000-of-00010
-rw-r--r-- 1 root root 3357425 Feb  5 17:57 fashion_mnist-train.tfrecord-00001-of-00010
-rw-r--r-- 1 root root 3371467 Feb  5 17:57 fashion_mnist-train.tfrecord-00002-of-00010
-rw-r--r-- 1 root root 3362288 Feb  5 17:57 fashion_mnist-train.tfrecord-00003-of-00010
-rw-r--r-- 1 root root 3380831 Feb  5 17:57 fashion_mnist-train.tfrecord-00004-of-00010
-rw-r--r-- 1 root root 3384792 Feb  5 17:57 fashion_mnist-train.tfrecord-00005-of-00010
-rw-r--r-- 1 root root 3355384 Feb  5 17:57 fashion_mnist-train.tfrecord-00006-of-00010
-rw-r--r-- 1 root root 3369155 Feb  5 17:57 fashion_mnist-train.tfrecord-00007-of-00010
-rw-r--r-- 1 root root 3361331 Feb  5 17

**TODO**
- ensure that cache works only for the same split

In [0]:
cat /root/tensorflow_datasets/fashion_mnist/1.0.0/dataset_info.json

{
  "citation": "@article{DBLP:journals/corr/abs-1708-07747,\n  author    = {Han Xiao and\n               Kashif Rasul and\n               Roland Vollgraf},\n  title     = {Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning\n               Algorithms},\n  journal   = {CoRR},\n  volume    = {abs/1708.07747},\n  year      = {2017},\n  url       = {http://arxiv.org/abs/1708.07747},\n  archivePrefix = {arXiv},\n  eprint    = {1708.07747},\n  timestamp = {Mon, 13 Aug 2018 16:47:27 +0200},\n  biburl    = {https://dblp.org/rec/bib/journals/corr/abs-1708-07747},\n  bibsource = {dblp computer science bibliography, https://dblp.org}\n}\n",
  "description": "Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.",
  "downloadChecksums": {
    "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-i

In [0]:
dataset['test']

<_OptionsDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>

## Back to the model

In [0]:
train_dataset, test_dataset = dataset["train"], dataset["test"]
assert isinstance(train_dataset, tf.data.Dataset)

# And then the rest of your input pipeline
train_dataset = train_dataset.repeat().shuffle(1024).batch(128).prefetch(2)
features = tf.compat.v1.data.make_one_shot_iterator(train_dataset).get_next()
image, label = features['image'], features['label']

In [0]:
test_features = tf.compat.v1.data.make_one_shot_iterator(test_dataset).get_next()
test_image, test_label = features['image'], features['label']

In [0]:
train_dataset.take(1)

<TakeDataset shapes: {image: (None, 28, 28, 1), label: (None,)}, types: {image: tf.uint8, label: tf.int64}>

In [0]:
features = tf.compat.v1.data.make_one_shot_iterator(train_dataset).get_next()
image, label = features['image'], features['label']


In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(image, label, epochs=5, steps_per_epoch=1000)


Epoch 1/5
1000/1000==============================] - 11s 11ms/step - loss: 1.2177 - acc: 0.9858
Epoch 2/5
1000/1000==============================] - 11s 11ms/step - loss: 0.0931 - acc: 0.9946
Epoch 3/5
1000/1000==============================] - 10s 10ms/step - loss: 0.0982 - acc: 0.9952
Epoch 4/5
1000/1000==============================] - 11s 11ms/step - loss: 0.0899 - acc: 0.9946
Epoch 5/5
1000/1000==============================] - 11s 11ms/step - loss: 0.0771 - acc: 0.9954


In [0]:
model.evaluate(test_image, test_label)

128/128==============================] - 0s 1ms/sample - loss: 15.8821 - acc: 0.7969


[15.882110357284546, 0.796875]